In [4]:
import zipfile
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import requests
import xml.etree.ElementTree as ET
import tensorflow as tf
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec

# Load and preprocess the data
csv_file = '/home/abarovic/laneyREUTest/eclipse_platform.csv'
full_df = pd.read_csv(csv_file)

full_df.fillna(0, inplace=True)
length = len(full_df)
split_index = int(.8 * length)
df = full_df.iloc[:split_index]  # Training set

df['full_text_data'] = df['Description'].fillna('') + ' ' + df['Title'].fillna('')

def is_duplicate(val):
    return 1 if val != 0.0 else 0

df['isDuplicate'] = df['Duplicated_issue'].apply(lambda x: is_duplicate(x))

# Tokenize the text data
def tokenize(text):
    return word_tokenize(text.lower())

df['tokenized_text'] = df['full_text_data'].apply(tokenize)

# Split the data
X = df['tokenized_text']  # Tokenized input features
y = df['isDuplicate']     # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Train a Word2Vec model
w2v_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Helper function to convert a list of words into a single vector
def vectorize_text(tokens, model):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Vectorize the training and testing data
X_train_vectors = np.array([vectorize_text(tokens, w2v_model) for tokens in X_train])
X_test_vectors = np.array([vectorize_text(tokens, w2v_model) for tokens in X_test])

# Fit the Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(X_train_vectors, y_train)

# Predict using the validation data
y_pred = gnb.predict(X_test_vectors)

# Calculate and print the classification report
report = classification_report(y_test, y_pred)
print(report)


/tmp/ipykernel_3432728/3367126447.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text_data'] = df['Description'].fillna('') + ' ' + df['Title'].fillna('')
/tmp/ipykernel_3432728/3367126447.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isDuplicate'] = df['Duplicated_issue'].apply(lambda x: is_duplicate(x))
/tmp/ipykernel_3432728/3367126447.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

ValueError: Negative values in data passed to MultinomialNB (input X)